# GitUp

In [ ]:
!git add .
!git commit -m "Graphs done pretty much"
!git push

In [2]:
# Libraries
from datashop import *
from scripts import *
import requests
import sqlite3
import pprint as pp
import pdb
from statsmodels.tsa.seasonal import seasonal_decompose
dep = Depot()
import plotly.offline as pyo
import plotly.graph_objs as go
from pmdarima.arima import auto_arima


from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.stattools import adfuller,kpss,coint,bds,q_stat,grangercausalitytests,levinson_durbin


In [3]:
daily_price = EIA_Series('Daily Price','PET.RWTC.D')
dep.ingest(daily_price)

desc = ''
daily_production = EIA_Series('Weekly Stocks','PET.WTTSTUS1.W')
dep.ingest(daily_production)

desc = 'US imports of crude oil, monthly'

monthly_imports = EIA_Series('Monthly Imports','PET.MCRIMUS1.M',desc,date_format='%Y%m' )
dep.ingest(monthly_imports)

weekly_sales = EIA_Series('Product Sold','PET.WRPUPUS2.W')
dep.ingest(weekly_sales)

conn = sqlite3.connect('data/energydash.db')
c = conn.cursor()

df = dep.originals['2000-01-01':]
df.columns = [x.replace(' ','_') for x in df.columns]
df.to_sql("original",conn,if_exists='replace')

In [4]:
df_news = pd.read_csv('data/nyt_articles.csv')
df_news = df_news[['date','abstract','main_headline','url']]
df_news['date_col'] = df_news['date'].str.slice(stop=10)
df_news['date_col']=pd.to_datetime( 
            df_news['date_col'],
            format = '%Y-%m-%d'
        )
display(report(df_news,10))
df_news['date_col'] = df_news['date_col'].sort_values()

conn = sqlite3.connect('data/energydash.db')
c = conn.cursor()

df_news.columns = [x.replace(' ','_') for x in df_news.columns]
df_news.to_sql("news_only",conn,if_exists='replace')

,type,% empty,unique values,1nth_value,1nth_value_%,2nth_value,2nth_value_%,3nth_value,3nth_value_%,4nth_value,...,6nth_value,6nth_value_%,7nth_value,7nth_value_%,8nth_value,8nth_value_%,9nth_value,9nth_value_%,10nth_value,10nth_value_%
main_headline,object,0.02,11479,THE MARKETS: COMMODITIES,1.69,Corrections,0.70,World Business Briefing,0.25,The Latest on the Oil Spill,...,BUSINESS DIGEST,0.06,"Driving Along, Less Gas in the Car",0.06,Gas Prices: Do Something Real,0.06,MEMO PAD,0.06,Trying to Wean America From Oil,0.06
abstract,object,0.56,12176,To the Editor:.,0.76,To the Editor:,0.12,World Business Briefing,0.11,By Bloomberg News.,...,EUROPE,0.05,In an administration dominated by oilmen and d...,0.04,Port for Long Island Sound Captain Peter Boynt...,0.03,>,0.03,The latest on stock market and business news d...,0.03
date_col,datetime64[ns],0.00,5109,2010-05-29 00:00:00,0.17,2008-06-29 00:00:00,0.17,2006-04-27 00:00:00,0.14,2010-06-19 00:00:00,...,2006-05-07 00:00:00,0.14,2010-06-15 00:00:00,0.14,2001-05-18 00:00:00,0.13,2010-06-17 00:00:00,0.13,2010-05-01 00:00:00,0.12
date,object,0.00,8074,2008-06-29T04:00:00+0000,0.15,2006-04-27T05:00:00+0000,0.13,2001-05-18T05:00:00+0000,0.13,2006-05-07T05:00:00+0000,...,2000-09-22T05:00:00+0000,0.10,2006-04-30T05:00:00+0000,0.10,2006-02-07T05:00:00+0000,0.10,2000-10-17T05:00:00+0000,0.09,2006-05-01T05:00:00+0000,0.09
url,object,0.00,12479,https://www.nytimes.com/2005/06/08/politics/bu...,0.02,https://www.nytimes.com/2010/05/13/us/13spill....,0.02,https://www.nytimes.com/2002/03/26/business/sh...,0.02,https://www.nytimes.com/2010/03/20/business/en...,...,https://www.nytimes.com/2000/05/30/nyregion/ga...,0.01,https://www.nytimes.com/2008/07/20/us/20patrol...,0.01,https://www.nytimes.com/2009/10/15/business/en...,0.01,https://www.nytimes.com/2016/05/09/business/sa...,0.01,https://www.nytimes.com/2006/10/20/world/asia/...,0.01


In [5]:
new_df = pd.merge(
                df_news,
                df,
                left_on='date_col',
                right_index=True
                )
new_df.head()

,date,abstract,main_headline,url,date_col,Daily_Price,Weekly_Stocks,Monthly_Imports,Product_Sold
2,2000-12-29T05:00:00+0000,Heating oil futures fall after report showing ...,THE MARKETS: COMMODITIES,https://www.nytimes.com/2000/12/29/business/th...,2000-12-29,26.72,1455285.0,286114.0,21494.0
12,2000-12-29T05:00:00+0000,Op-Ed article by former Pres Jimmy Carter desc...,Make This Natural Treasure a National Monument,https://www.nytimes.com/2000/12/29/opinion/mak...,2000-12-29,26.72,1455285.0,286114.0,21494.0
4,2000-12-27T05:00:00+0000,Heating oil prices for January delivery soar m...,THE MARKETS: COMMODITIES,https://www.nytimes.com/2000/12/27/business/th...,2000-12-27,26.55,1466328.0,286114.0,20080.0
6,2000-12-27T05:00:00+0000,Russian government plans to sell 50 million sh...,AMERICAN SHARE SALE FOR LUKOIL,https://www.nytimes.com/2000/12/27/business/wo...,2000-12-27,26.55,1466328.0,286114.0,20080.0
8,2000-12-27T05:00:00+0000,Government reports that Vietnam's economy grew...,Vietnam's Trade Imbalance And Economy Keep Gro...,https://www.nytimes.com/2000/12/27/business/vi...,2000-12-27,26.55,1466328.0,286114.0,20080.0


In [6]:
conn = sqlite3.connect('data/energydash.db')
c = conn.cursor()

new_df.columns = [x.replace(' ','_') for x in new_df.columns]

new_df.to_sql("news_merged",conn,if_exists='replace')

# Daily Price

In [ ]:
desc = ''
daily_price = EIA_Series('Daily Price','PET.RWTC.D')
dep.ingest(daily_price)

x = daily_price.scaled
#x = x['2015-01-01':'2019-12-01']
result = seasonal_decompose(x, model='additive',period=365)  
result.plot();

In [ ]:
from statsmodels.tsa.stattools import adfuller
dftest = adfuller(x,autolag='AIC')
print(dftest)

## Auto Arima

In [ ]:
auto_arima(x,start_p=1,start_q=1,max_p = 10,max_q=10,seasonal=True,trace=True,m=365)

In [ ]:
span = 365
alpha = 2/(span+1) # Standard span formula



model =SimpleExpSmoothing(x)

fitted_model = model.fit(
    smoothing_level=alpha,
    optimized=False)
fitted_model = model.fit()

daily_price.frame['SimpleExp'] = fitted_model.fittedvalues.shift(-1) 

df_slice = daily_price.frame[['SimpleExp','scaled_Daily Price']]

df_slice['2016-01-01':'2016-06-01'].plot(figsize=(10,20))




# ARIMA

# Daily Production

In [ ]:
desc = ''
daily_production = EIA_Series('Weekly Stocks','PET.WTTSTUS1.W')
dep.ingest(daily_production)

x2 = daily_production.scaled
x2 = x2['2015-01-01':'2019-12-01']
result = seasonal_decompose(x2, model='additive',period=7)  
result.plot();

In [ ]:
granger_frame = pd.concat([x,x2],axis=1)
granger_frame.columns = ['price','stocks']
granger_frame.head()
g_test = grangercausalitytests(granger_frame,maxlag=90);

In [ ]:
print(g_test[1])

In [ ]:
import chart_studio.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objects as go
import pandas as pd
import plotly

y = x.index

trace1 = go.Scatter(
        x=y,
        y=x,
        mode='lines',
        name='Spot Price'
)

trace2 = go.Scatter(
    x=x2.index,
    y=x2,
    mode='lines',
    name = 'Weekly Inventory')

data = [trace1,trace2]

layout = go.Layout(
    title = 'Daily Stocks',
    xaxis = {
        'title':'Time',
    },
    yaxis = dict(
        title='Scaled price'
        ),
    hovermode='closest'
)

fig = go.Figure(data=data, layout = layout)

plotly.offline.iplot(fig,filename='daily_price')

# Monthly Crude Imports

In [ ]:
desc = 'US imports of crude oil, monthly'

monthly_imports = EIA_Series('Monthly Imports','PET.MCRIMUS1.M',desc,date_format='%Y%m' )
dep.ingest(monthly_imports)

x = monthly_imports.scaled
x = x['2000-01-01':]

result = seasonal_decompose(x, model='additive',period=12)  
result.plot();


In [ ]:
ds = Dataset(dep.originals)

In [ ]:
ds['Monthly Imports'].show_report()

## Dow Jones

In [ ]:
df_dow = pd.read_csv('data/dji.csv')

df_dow['Date']=pd.to_datetime(df_dow['Date'])
        
df_dow.set_index(df_dow['Date'],drop=True,inplace=True)

df_dow.columns

In [ ]:
desc = 'US imports of crude oil, monthly'

x = df_dow['Close']
x = x['2000-01-01':]

result = seasonal_decompose(x, model='additive',period=365)  
result.plot();

In [ ]:
df_dow['Close'].plot()

In [ ]:
df_dow['delta_dow']  = df_dow['Close'].diff()

X = df_dow[['delta_dow']]

y = daily_price.series_frame[['deltaDaily Price']]

merged = y.merge(X,how='inner',left_index=True,right_index=True)

In [ ]:
merged.head()

In [ ]:
x=merged['deltaDaily Price']
y=merged['delta_dow']
plt.scatter(x=x,y=y)

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

model= LinearRegression()

In [ ]:
model.fit(x.reshape(-1,1),y)

In [ ]:
dep.scaled.head(10)

# To-Do

1. Moving averages?
2. Correlation between price and:
3. Features to add/explore:
    a. Active wells?
    b. Refinery inputs?
    c. consumption?
    d. month?
    e. 
    


# SCRAPS

In [ ]:
\
df_eia=functools.reduce(lambda x,y: x.join(y,how='outer'),frame_list)
df_eia.fillna(axis=0,method='ffill',inplace=True)

conn = sqlite3.connect('oilstocks.db')
c = conn.cursor()

for col in df_eia:
    df_eia[col]=preprocessing.scale(df_eia[col])

df_eia.to_sql("scaledstocks",conn,if_exists='replace')
conn.commit()

df_eia.to_sql("crudestocks",conn,if_exists='replace')